In [2]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers, training_functions
import config

window_size = 96
pred_length = (96)#, 192, 336, 720)

print("Import succesfull")


Import succesfull


# Select dataset for training

In [2]:
# eu electricity data
data_tensor = data_handling.eu_electricity_to_tensor()
data_dict, standardize_values =  data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to dataloader
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 208])


In [8]:
# electricity dataset
data_dict = data_handling.load_electricity()

# convert to dataloader
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)


Feature batch shape: torch.Size([32, 96, 348])


131

In [3]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()

data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

len(dataloader_train)

Feature batch shape: torch.Size([32, 96, 67])


52

# Train model on selected dataset

In [3]:
# for electricity / eu electricity
#best_parameters = {'depth': 3, 'dim': 818, 'dim_head': 58, 'heads': 4, 'attn_dropout': 0.17358586508257956, 'ff_mult': 4,
 #                'ff_dropout': 0.07147101457460232, 'num_mem_tokens': 4, 'learning_rate': 0.0008061714887745944}

#paper_parameters = {'depth': 2, 'dim': 256, 'learning_rate': 0.0005}

best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                   'num_mem_tokens': 4, 'learning_rate': 0.0005}

model_config = {
    'num_variates': data_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': best_parameters["depth"],
    'dim': best_parameters["dim"],
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': best_parameters["dim_head"],
    'heads': best_parameters["heads"],
    'attn_dropout': best_parameters["attn_dropout"],
    'ff_mult': best_parameters["ff_mult"],
    'ff_dropout': best_parameters["ff_dropout"],
    'num_mem_tokens': best_parameters["num_mem_tokens"],
    'use_reversible_instance_norm': False,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# defining all needed instances
model = iTransformer(**model_config).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["revin"])


Using device: cuda
Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [6]:
epoch = 11

for epoch in range(1, epoch + 1):
	training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)



Epoch: 1: 100%|██████████| 131/131 [00:10<00:00, 12.26it/s]


Epoch 1, MSE-Loss: 0.3245384626033652, LR: 0.0005


Epoch: 2: 100%|██████████| 131/131 [00:09<00:00, 13.34it/s]


Epoch 2, MSE-Loss: 0.21381669497216932, LR: 0.0005


Epoch: 3: 100%|██████████| 131/131 [00:09<00:00, 13.55it/s]


...

Epoch: 10: 100%|██████████| 131/131 [00:08<00:00, 16.01it/s]


Epoch 10, MSE-Loss: 0.16684571010920837, LR: 5e-05
Checkpointing succesfull after epoch 10


Epoch: 11: 100%|██████████| 131/131 [00:08<00:00, 14.87it/s]


Epoch 11, MSE-Loss: 0.16598104850481485, LR: 5e-06


Epoch: Validating: 100%|██████████| 1/1 [00:00<00:00, 31.82it/s]


In [10]:
metrics = training_functions.fast_eval(model, dataloader_test)
metrics

Epoch: Validating: 100%|██████████| 67/67 [00:02<00:00, 27.98it/s]


{96: {'mse': tensor(0.1657, device='cuda:0'),
  'mae': tensor(0.2497, device='cuda:0'),
  'smape': tensor(105.7122, device='cuda:0'),
  'p10': tensor(0.0286, device='cuda:0'),
  'p50': tensor(0.1653, device='cuda:0'),
  'p90': tensor(0.5340, device='cuda:0')},
 192: {'mse': tensor(0.1915, device='cuda:0'),
  'mae': tensor(0.2865, device='cuda:0'),
  'smape': tensor(116.4241, device='cuda:0'),
  'p10': tensor(0.0336, device='cuda:0'),
  'p50': tensor(0.1921, device='cuda:0'),
  'p90': tensor(0.6345, device='cuda:0')},
 336: {'mse': tensor(0.2228, device='cuda:0'),
  'mae': tensor(0.3165, device='cuda:0'),
  'smape': tensor(122.2620, device='cuda:0'),
  'p10': tensor(0.0365, device='cuda:0'),
  'p50': tensor(0.2094, device='cuda:0'),
  'p90': tensor(0.7290, device='cuda:0')},
 720: {'mse': tensor(0.4058, device='cuda:0'),
  'mae': tensor(0.4179, device='cuda:0'),
  'smape': tensor(141.5883, device='cuda:0'),
  'p10': tensor(0.0432, device='cuda:0'),
  'p50': tensor(0.2558, device='cuda:0

In [23]:
# predict on eu electricity
metrics = training_functions.fast_eval(dataloader_test)
metrics

Epoch: Validating: 100%|██████████| 248/248 [00:08<00:00, 28.32it/s]

Pred_len 96 MSE:  tensor(1.0267, device='cuda:0')


{96: {'mse': tensor(1.0267, device='cuda:0'),
  'mae': tensor(0.6372, device='cuda:0'),
  'smape': tensor(157.1326, device='cuda:0'),
  'p10': tensor(0.0509, device='cuda:0'),
  'p50': tensor(0.3867, device='cuda:0'),
  'p90': tensor(1.4745, device='cuda:0')},
 192: {'mse': tensor(1.6828, device='cuda:0'),
  'mae': tensor(0.8021, device='cuda:0'),
  'smape': tensor(170.9238, device='cuda:0'),
  'p10': tensor(0.0637, device='cuda:0'),
  'p50': tensor(0.4290, device='cuda:0'),
  'p90': tensor(2.1294, device='cuda:0')},
 336: {'mse': tensor(1.9419, device='cuda:0'),
  'mae': tensor(0.8371, device='cuda:0'),
  'smape': tensor(170.4025, device='cuda:0'),
  'p10': tensor(0.0720, device='cuda:0'),
  'p50': tensor(0.4566, device='cuda:0'),
  'p90': tensor(2.1610, device='cuda:0')},
 720: {'mse': tensor(2.1341, device='cuda:0'),
  'mae': tensor(0.8491, device='cuda:0'),
  'smape': tensor(180.1998, device='cuda:0'),
  'p10': tensor(0.0843, device='cuda:0'),
  'p50': tensor(0.4829, device='cuda:0

# Load saved model and predict on specified dataset

We load the model trained for 10 epoch on the original electricity dataset.

-Try zero shot prediction on the test dataset

-Re-train on train or validation dataset

-Test again on test dataset



In [6]:
# use electricity dataset
#data_dict = data_handling.load_electricity()

# train for eu_electricity
#for key, values in data_dict.items():
#    data_dict[key] = values[:, 0:208]

#dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

# for electricity
#best_parameters = {'depth': 3, 'dim': 818, 'dim_head': 58, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
 #                  'num_mem_tokens': 4, 'learning_rate': 0.0005}

best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                   'num_mem_tokens': 4, 'learning_rate': 0.0005}

model_config = {
    'num_variates': data_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': best_parameters["depth"],
    'dim': best_parameters["dim"],
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': best_parameters["dim_head"],
    'heads': best_parameters["heads"],
    'attn_dropout': best_parameters["attn_dropout"],
    'ff_mult': best_parameters["ff_mult"],
    'ff_dropout': best_parameters["ff_dropout"],
    'num_mem_tokens': best_parameters["num_mem_tokens"],
    'use_reversible_instance_norm': False,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = iTransformer(**model_config).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["revin"])

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [7]:
model_path = "/vol/fob-vol7/nebenf21/reinbene/bene/MA/outputs/models/revin/trial_epoch_10_loss_0.6577947735786438.pt"
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])

#model.load_state_dict(model.state_dict())


<All keys matched successfully>

In [10]:
metrics = training_functions.fast_eval(model, dataloader_test)
metrics

Epoch: Validating: 100%|██████████| 52/52 [00:01<00:00, 30.91it/s]


{96: {'mse': tensor(3.6800, device='cuda:0'),
  'mae': tensor(1.7944, device='cuda:0'),
  'smape': tensor(180.0295, device='cuda:0'),
  'p10': tensor(0.9925, device='cuda:0'),
  'p50': tensor(1.7757, device='cuda:0'),
  'p90': tensor(2.5852, device='cuda:0')},
 192: {'mse': tensor(3.7270, device='cuda:0'),
  'mae': tensor(1.8051, device='cuda:0'),
  'smape': tensor(179.4368, device='cuda:0'),
  'p10': tensor(1.0039, device='cuda:0'),
  'p50': tensor(1.7916, device='cuda:0'),
  'p90': tensor(2.6064, device='cuda:0')},
 336: {'mse': tensor(3.6725, device='cuda:0'),
  'mae': tensor(1.7834, device='cuda:0'),
  'smape': tensor(173.9219, device='cuda:0'),
  'p10': tensor(0.9526, device='cuda:0'),
  'p50': tensor(1.7725, device='cuda:0'),
  'p90': tensor(2.6025, device='cuda:0')},
 720: {'mse': tensor(3.3147, device='cuda:0'),
  'mae': tensor(1.6667, device='cuda:0'),
  'smape': tensor(153.6199, device='cuda:0'),
  'p10': tensor(0.7585, device='cuda:0'),
  'p50': tensor(1.6460, device='cuda:0

In [11]:
epoch = 1
train_one_epoch(epoch, model, device, dataloader_validation, dataloader_test, optimizer, scheduler, writer)


NameError: name 'train_one_epoch' is not defined